In [ ]:
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        prev = 0
        loss = model.get_latest_training_loss()
        change_loss = loss - prev
        prev = loss
        print('Loss after epoch {}: {}'.format(self.epoch, change_loss))
        self.epoch += 1

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-05-25 02:48:29,597 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2024-05-25 02:48:29,603 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2024-05-25 02:48:29,605 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2024-05-25T02:48:29.605889', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-5.15.146.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'created'}


Define the corpus, cleaning for punctuation, tags and multiple whitespaces
Shuffe the corpus
Yield each sentence of the shuffled corpus

In [ ]:
from gensim.test.utils import datapath
import gensim.parsing.preprocessing as gsp
import random

class EventCorpus:
	"""An iterator that yields sentences (lists of str) for pseudo event corpus."""
	def __iter__(self):
		sentences = []
		corpus_path = datapath('/home/vlnsha004/CSC2005Z/lpe_soccerevents/data/eventcorpus8.txt')
		
		for line in open(corpus_path):

			# remove all single quotation marks
			line = line.replace("'", "") # Erroneous quotation marks
			line = line.replace('6', '5')
			line = line.replace('"', '')
			CUSTOM_FILTERS = [gsp.strip_tags, gsp.strip_multiple_whitespaces]
			sentences.append(gsp.preprocess_string(line, CUSTOM_FILTERS))
			yield gsp.preprocess_string(line, CUSTOM_FILTERS)
        
		random.shuffle(sentences)  # Shuffle the sentences

		for sentence in sentences:
			yield sentence

Model training
Current hyperparameters:
window = 3
epochs = 35
vs = 500
mc = 1 # min_count
negative = 20, Negative used for the sample size used for negative sampling

In [ ]:
from gensim.models import Word2Vec

# Train the model on the new data
# model.train(sequences, total_examples=model.corpus_count, epochs=model.epochs, compute_loss = True, callbacks=[callback()])

sequences = EventCorpus()

window = 10
epochs = 60
vs = 800
mc = 1 # min_count
ns = 20 # negative sampling
subsample = 1e-5
model = Word2Vec(sentences=sequences, epochs = epochs, sample=subsample, vector_size=vs, window=window, min_count=mc, workers=10, compute_loss = True, negative=ns, callbacks=[callback()]) #Obtain params from football2vec text
model.save(f"/home/vlnsha004/CSC2005Z/lpe_soccerevents/models/anon/Seq2vec_{epochs}_{vs}_{window}_{mc}_{ns}_shuffled_sample.model") # First model trained on full corpus

2024-05-25 02:49:50,824 : INFO : collecting all words and their counts
2024-05-25 02:49:50,866 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2024-05-25 02:49:51,696 : INFO : PROGRESS: at sentence #10000, processed 58929 words, keeping 147 word types
2024-05-25 02:49:52,586 : INFO : PROGRESS: at sentence #20000, processed 117432 words, keeping 191 word types
2024-05-25 02:49:53,296 : INFO : PROGRESS: at sentence #30000, processed 175749 words, keeping 225 word types
2024-05-25 02:49:53,893 : INFO : PROGRESS: at sentence #40000, processed 234430 words, keeping 276 word types
2024-05-25 02:49:54,655 : INFO : PROGRESS: at sentence #50000, processed 293093 words, keeping 322 word types
2024-05-25 02:49:55,322 : INFO : PROGRESS: at sentence #60000, processed 351564 words, keeping 347 word types
2024-05-25 02:49:56,062 : INFO : PROGRESS: at sentence #70000, processed 410018 words, keeping 361 word types
2024-05-25 02:49:57,009 : INFO : PROGRESS: at sentence #80000

: 

In [ ]:
sequences = EventCorpus()
window = 3
epochs = 30
vs = 500
mc = 1 # min_count
ns = 5 # negative sampling
for i in range(6, 18, 6):
    window = i
    for j in range(5, 25, 15):
        ns = j
        model = Word2Vec(sentences=sequences, epochs = epochs, vector_size=vs, window=window, min_count=mc, workers=8, compute_loss = True, negative=ns, callbacks=[callback()])
        model.save(f"/home/vlnsha004/CSC2005Z/player2vec/models/anon/Seq2vec_{epochs}_{vs}_{window}_{mc}_{ns}_unshuffled.model") # First model trained on full corpus

In [ ]:
model.save(f"/home/vlnsha004/CSC2005Z/lpe_soccerevents/models/anon/Seq2vec_{epochs}_{vs}_{window}_{mc}_{ns}_unshuffled.model") # First model trained on full corpus

2024-05-23 08:28:21,072 : INFO : Word2Vec lifecycle event {'fname_or_handle': '/home/vlnsha004/CSC2005Z/lpe_soccerevents/models/anon/Seq2vec_30_500_3_1_5_unshuffled.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2024-05-23T08:28:21.072336', 'gensim': '4.3.2', 'python': '3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]', 'platform': 'Linux-5.15.146.1-microsoft-standard-WSL2-x86_64-with-glibc2.35', 'event': 'saving'}
2024-05-23 08:28:21,075 : INFO : not storing attribute cum_table
2024-05-23 08:28:21,289 : INFO : saved /home/vlnsha004/CSC2005Z/lpe_soccerevents/models/anon/Seq2vec_30_500_3_1_5_unshuffled.model


In [8]:
from gensim.models import Word2Vec

model_path = "/home/vlnsha004/CSC2005Z/lpe_soccerevents/models/anon/Seq2vec_30_500_3_1_20_shuffled.model"
model = Word2Vec.load(model_path)

In [2]:
import pandas as pd

fifa =  pd.read_csv('/home/vlnsha004/CSC2005Z/lpe_soccerevents/data/male_players.csv', delimiter=',', low_memory=False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [3]:
import dask.dataframe as dd
import multiprocessing as mp

fifa = fifa.query('fifa_version > 23.0')
fifa = fifa.sort_values(['overall', 'player_positions'], ascending = False)
fifa['long_name'] = fifa['long_name'].str.replace(' ', '_')

fifa = fifa.reset_index(drop=True, inplace=False) # reset the indices
fifa = fifa.rename_axis("index", axis="columns")
trimmed_fifa = fifa[["long_name", "player_positions", "player_tags", "player_traits"]]
take_sample_400 = trimmed_fifa.sample(n=400, random_state=1, replace=False)
top400 = dd.from_pandas(take_sample_400, npartitions=mp.cpu_count()) #Take 400 of the top rated players

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def compare(baller, sim_player, pd_fifa, trials, type) -> int:
	'''type -> compare positions or traits or tags'''
	'''type: string representing column name '"player_positions", "player_tags", "player_traits"'''

	baller_positions = (pd_fifa.loc[pd_fifa['long_name'] == baller, type]).unique().tolist()
	if len(baller_positions) == 0 or str(baller_positions[0]) == 'nan':
		print(f'{baller} has no player tags in FIFA\n')
		trials = 1
		print(f'Trials = {trials}\n')
		return [0, trials]
	baller_set = set(baller_positions[0].split(', '))
	# Return 1 if sim_player plays any of the baller's positions, 0 otherwise
	# Find rows where sim_player is a subset of long_name efficiently with vectorization
	subset_mask = pd_fifa['long_name'].str.contains(sim_player[0], case=False, regex=False)  # Case-insensitive
	# Assuming there's only one matching record (use .iloc[0] for the first)
	matching_row = pd_fifa[subset_mask]
	retrieval_w2v_positions = matching_row[type].unique().tolist()
	if len(retrieval_w2v_positions) == 0 or 'nan' in (str(retrieval_w2v_positions)):
		print(f'{sim_player[0]} not found or has no traits in FIFA\n')
		print(f'Trials = {trials}\n')
		return [0, trials]
	elif len(retrieval_w2v_positions) > 1:
		print(f'{sim_player[0]}: {retrieval_w2v_positions}')
		retrieval_set = set(retrieval_w2v_positions[1].split(', '))
		print(f'{baller}: {baller_positions}')    
		print(f'similar player set: {retrieval_set}')
		print(f'baller set: {baller_set}')
		print(f'intersection: {retrieval_set & baller_set}')
		if (retrieval_set & baller_set):
			# print('SIMILAR PLAYER FOUND\n')
			trials += 1
			return [1, trials]
		else:
			# print('No match\n')
			trials += 1
			return [0, trials]
	else:
		print(f'{sim_player[0]}: {retrieval_w2v_positions}')
		retrieval_set = set(retrieval_w2v_positions[0].split(', '))
		print(f'{baller}: {baller_positions}')    
		print(f'similar player set: {retrieval_set}')
		print(f'baller set: {baller_set}')
		print(f'intersection: {retrieval_set & baller_set}')
		if (retrieval_set & baller_set):
			print('SIMILAR PLAYER FOUND\n')
			trials += 1
			return [1, trials]
		else:
			print('No match\n')
			trials += 1
			return [0, trials]
	

In [11]:
import random
import numpy as np

def get_pakvector(top400, base=False, type="player_positions", model=model):

	similar_players = {}

	for player in top400['long_name']:
		# Create dictionary with {"Eriksen": ["Davies", "Mignolet", "Ronaldo"], "Johnathan": ["Bakambu", "Cedric"]}
		try: 
			if base:
				all_players = model.wv.index_to_key
				random_players = random.sample(all_players, 200)
				similar_players[player] = [(other_player, model.wv.similarity(player, other_player)) for other_player in random_players]
			else:
				similar_players[player] = model.wv.most_similar(player, topn=60)
		except KeyError:
			continue
		
	# Create an array to store the precision at 20 for each player in selected_columns, retrieving similar players by the model
	pak_store = []
	for baller, similar in similar_players.items(): # Loop through the key-value pairs
		counter = 0
		trials = 0
		print(f'{baller}: {similar}')
		baller_positions = (trimmed_fifa.loc[trimmed_fifa['long_name'] == baller, type]).unique().tolist()
		if len(baller_positions) == 0 or 'nan' in (str(baller_positions)):
			print(f'{baller} has no player tags in FIFA\n')
			continue
		# Begin trials
		for sim_player in similar: # For each player in similar set:	
			[match, new_trial] = compare(baller, sim_player, trimmed_fifa, trials, type)
			counter += match
			trials = new_trial
			if trials == 10:
				prop = counter/trials
				break
		# If we do not have 10 trials, do not include in the precision at 10 calculation
		# Have gone through all retrievals and found no viable comparisons
		if trials < 10:
			print(f'Less than 10 data points for similar players to {baller} in FIFA\n')
			continue

		print(f'{baller}: Correct:{counter}, Trials:{trials}, Percent: {prop}\n')
		pak_store.append(prop) # Store the precision at 20 for each player
	ave_precision_k = round(np.mean(pak_store) * 100, 2)
	return ave_precision_k

In [ ]:
print(get_pakvector(top400, base=False, type="player_tags", model = model))

Harry_Kane: [('Andy_Delort', 0.976628839969635), ('Diego_da_Silva_Costa', 0.9762033820152283), ('Martin_Braithwaite_Christensen', 0.9750285744667053), ('Wissam_Ben_Yedder', 0.9740056395530701), ('Lucas_Pérez_Martínez', 0.9720433354377747), ('Federico_Dionisi', 0.9718557000160217), ('Borja_González_Tomás', 0.9714374542236328), ('Jamie_Vardy', 0.9710861444473267), ('Timo_Werner', 0.970790684223175), ('Haris_Seferović', 0.9700158834457397), ('Rubén_Castro_Martín', 0.9696327447891235), ('Jimmy_Briand', 0.96944260597229), ('Sergio_Floccari', 0.9693285822868347), ('Fernando_José_Torres_Sanz', 0.9690831303596497), ('Benjamin_Moukandjo_Bilé', 0.9682694673538208), ('Jacques_Zoua_Daogari', 0.9682202935218811), ('Cyril_Théréau', 0.968169093132019), ('Francisco_Alcácer_García', 0.9676008224487305), ('Youssef_El-Arabi', 0.9675054550170898), ('Michy_Batshuayi_Tunga', 0.9674092531204224), ('Álvaro_Borja_Morata_Martín', 0.967249870300293), ('Cédric_Bakambu', 0.9669193625450134), ('Giovanni-Guy_Yann_Si

In [13]:
import os
from gensim.models import Word2Vec

directory = "/home/vlnsha004/CSC2005Z/lpe_soccerevents/models/anon"
types = ["player_positions", "player_tags", "player_traits"]
models = {}

for filename in os.listdir(directory):
    if filename.endswith(".model"):
        model_path = os.path.join(directory, filename)
        model_name = filename[:-6]  # remove ".model" from the filename
        models[model_name] = Word2Vec.load(model_path)
        

latex_table = r"""
\begin{tabular}{|l||c|c||c|c|c||c|}
\hline
Model    &   Negative          & Anonymized/    & Shuffled/    & \multicolumn{3}{c||}{Accuracy [\%]} \\
         &   Sample  & Unanonymized   & Unshuffled   & \multicolumn{3}{c||}{Precision at K} \\
\hline
         &                             &       &    & Player Positions      &  Player Tags    &  Player Traits               \\
\hline
"""

# Add the random/base set of results
latex_table += f"Randomly drawn set of retrievals & N/A & N/A & N/A & "
for type in types:
    pak = get_pakvector(top400, base = True, type=type)
    latex_table += f"{pak} & "
latex_table = latex_table[:-2]  # remove the last "& "
latex_table += r"\\ \hline"  # add a new row

# Define the sorting key
def sorting_key(item):
    model_name, model = item
    shuffled = "shuffled" in model_name
    return (shuffled, model.negative)

# Sort the models
sorted_models = sorted(models.items(), key=sorting_key)

for model_name, model in sorted_models:
    anonymized = "Anon" if "anon" in model_name else "Unanon"
    shuffled = "Shuffled" if "shuffled" in model_name else "Unshuffled"
    latex_table += f"{model_name} & {model.negative} & {anonymized} & {shuffled} & "
    for type in types:
        pak = get_pakvector(top400, type=type, model = model)
        latex_table += f"{pak} & "
    latex_table = latex_table[:-2]  # remove the last "& "
    latex_table += r"\\ + \n+ \hline + \n"  # add a new row

latex_table += r"\end{tabular}"

print(latex_table)

Harry_Kane: [('Nayef_Aguerd', 0.32586235), ('Harry_Kewell', 0.82815635), ('Maximilian_Wöber', 0.281113), ('Paul_Verhaegh', 0.58108693), ('Juan_Fernando_Arango_Sáenz', 0.84183043), ('Lost', -0.20940556), ('Dejalma_Pereira_Dias_dos_Santos', 0.35058343), ('Geny_Cipriano_Catamo', 0.54703826), ('Yohann_Pelé', -0.17767616), ('Sebastian_Rudy', 0.82110214), ('Dieter_Hoeneß', 0.30227056), ('Calvin_Bassey_Ughelumba', 0.4780471), ('Gorka_Elustondo_Urkola', 0.7708867), ('Christoph_Baumgartner', 0.7896218), ('Devis_Rogers_Epassy_Mboka', -0.13423115), ('David_Cortés_Caballero', 0.38224393), ('Fernando_Pacheco_Flores', 0.0057341866), ('Marc_Albrighton', 0.8769758), ('Aritz_Elustondo_Irribaria', 0.6018634), ('Borja_Valero_Iglesias', 0.8620315), ('Wim_Rijsbergen', 0.61484265), ('Dejan_Lovren', 0.5521258), ('Fredrik_Jensen', -0.12919937), ('Karol_Linetty', 0.6835771), ('Shot_Off_T', -0.06490907), ('Yoann_Court', 0.8783759), ('Kévin_Soni', 0.7441091), ('Yacouba_Nasser_Djiga', -0.10630123), ('Haji_Mnoga',

In [ ]:
# Evaluation and retrievals:

word = 'Lionel_Andrés_Messi_Cuccittini'
try:
	print(model.wv.most_similar(word, topn = 45))
except KeyError:
    print(f"The word {word} does not appear in current model") 



[('Xavier_Hernández_Creus', 0.9263545870780945), ('Francesc_Fàbregas_i_Soler', 0.9195429086685181), ('Ángel_Fabián_Di_María_Hernández', 0.9178848266601562), ('Alexis_Alejandro_Sánchez_Sánchez', 0.9170514941215515), ('Juan_Manuel_Mata_García', 0.916039228439331), ('Samuel_Etoo_Fils', 0.9155625700950623), ('Luis_Alberto_Suárez_Díaz', 0.9149684309959412), ('Pedro_Eliezer_Rodríguez_Ledesma', 0.9149653911590576), ('Fabián_Ariel_Orellana_Valenzuela', 0.9129148125648499), ('Antoine_Griezmann', 0.9126431941986084), ('Riccardo_Saponara', 0.9122297167778015), ('Ousmane_Dembélé', 0.9106715321540833), ('Paulo_Bruno_Exequiel_Dybala', 0.9078379273414612), ('Iago_Aspas_Juncal', 0.9065255522727966), ('Josip_Iličić', 0.9064397215843201), ('Zlatan_Ibrahimović', 0.9046342968940735), ('Hatem_Ben_Arfa', 0.9035216569900513), ('Ángel_Martín_Correa', 0.9012441039085388), ('Mesut_Özil', 0.9000458121299744), ('Miralem_Pjanić', 0.8956150412559509), ('Rafael_Alcântara_do_Nascimento', 0.8952606916427612), ('Thomas

In [ ]:
pairs = [
    ('Diego_Armando_Maradona', word),  # a minivan is a kind of car
    ('long_pass', word),   # a minivan is a kind of car
    ('High_Pass', word),   # still a wheeled vehicle
    ('short_pass', word),  # ok, no wheels, but still a vehicle
    ('Ronaldo_de_Assis_Moreira', word),    # ... and so on
    ('Shot_Goal', word),
]
for w1, w2 in pairs:
    print(f'{w1}\t{w2}\t{model.wv.similarity(w1, w2)}')
    # print(f'{w1}\t{w2}\t{model300.wv.similarity(w1.lower(), w2.lower())}')

Diego_Armando_Maradona	Lionel_Andrés_Messi_Cuccittini	0.8746740221977234
long_pass	Lionel_Andrés_Messi_Cuccittini	0.08580203354358673
High_Pass	Lionel_Andrés_Messi_Cuccittini	0.16584700345993042
short_pass	Lionel_Andrés_Messi_Cuccittini	0.23890675604343414
Ronaldo_de_Assis_Moreira	Lionel_Andrés_Messi_Cuccittini	0.8516892790794373
Shot_Goal	Lionel_Andrés_Messi_Cuccittini	-0.04527859762310982


In [ ]:
# Get the embeddings for three words
word2_vector = model.wv['Kylian_Mbappé_Lottin']
word3_vector = model.wv['Low_Pass']
word1_vector = model.wv['Interception']
word4_vector = model.wv['(4,4)']

# Perform vector operations
result_vector = word1_vector + word3_vector + word2_vector
# Get the words most similar to the result vector
most_similar_words = model.wv.most_similar(positive=[result_vector])

# Print the most similar words
for word, similarity in most_similar_words:
	print(word, similarity)

Low_Pass 0.8058099746704102
Ground_Pass 0.7488465905189514
High_Pass 0.6096218824386597
Interception 0.5788564085960388
ball_carry_to 0.5253614783287048
César_Azpilicueta_Tanco 0.43268486857414246
Jonathan_Castro_Otto 0.4264591634273529
Fabián_Ruiz_Peña 0.4250446557998657
Adriano_Correia_Claro 0.416988730430603
Sergio_Ramos_García 0.4157983660697937


In [ ]:
# Get vocab of model

w2v_vocabulary = model.wv.key_to_index
print((w2v_vocabulary))

{'Pass_to': 0, 'Pass_received_by': 1, 'Ground_Pass': 2, 'medium_pass': 3, 'ball_carry_to': 4, 'short_pass': 5, 'High_Pass': 6, '(3,1)': 7, 'long_pass': 8, '(3,5)': 9, '(3,2)': 10, '(3,4)': 11, '(3,3)': 12, '(4,5)': 13, '(4,1)': 14, '(2,2)': 15, '(2,4)': 16, '(4,4)': 17, '(4,2)': 18, 'Low_Pass': 19, '(2,1)': 20, '(2,3)': 21, '(2,5)': 22, '(1,3)': 23, '(4,3)': 24, '(5,5)': 25, '(5,1)': 26, 'ball_receive_failed': 27, '(5,3)': 28, '(5,2)': 29, '(5,4)': 30, '(1,4)': 31, '(1,2)': 32, '(1,1)': 33, '(1,5)': 34, 'Lionel_Andrés_Messi_Cuccittini': 35, 'clearance': 36, 'Cross': 37, 'Tackle': 38, 'Aerial_Lost': 39, 'Sergio_Busquets_i_Burgos': 40, 'Xavier_Hernández_Creus': 41, 'Andrés_Iniesta_Luján': 42, 'Head': 43, 'Interception': 44, 'Gerard_Piqué_Bernabéu': 45, 'Complete_dribble': 46, 'Jordi_Alba_Ramos': 47, 'Daniel_Alves_da_Silva': 48, 'Incomplete_dribble': 49, 'Ivan_Rakitić': 50, 'Right_Foot': 51, 'Duel_Success_In_Play': 52, 'Javier_Alejandro_Mascherano': 53, 'Neymar_da_Silva_Santos_Junior': 54